In [29]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# ÉTAPE 1 : Chargement du dataset

In [30]:
df = pd.read_csv("Decathlon_data.csv")

# Colonnes utiles (sku est obligatoire)
df = df[['sku', 'nom', 'category', 'marque', 'prix', 'note']]

print("Produits chargés :", len(df))

Produits chargés : 936


# ÉTAPE 2 : Nettoyage et conversion du prix

In [31]:
# Nettoyage de la colonne prix
df['prix'] = (
    df['prix']
    .astype(str)
    .str.replace('\u202f', '', regex=False)  # enlève les espaces insécables
    .str.replace('MAD', '', regex=False)
    .str.replace(',', '.', regex=False)
    .str.extract(r'(\d+\.?\d*)')[0]
)

# Conversion en float
df['prix'] = pd.to_numeric(df['prix'], errors='coerce')


# ÉTAPE 3 : Nettoyage léger des données

In [32]:
# Supprimer les lignes avec des valeurs manquants
df.dropna(inplace=True)

# ÉTAPE 4 : Suppression des doublons avec SKU

In [34]:
# Si un même produit apparaît plusieurs fois, on garde 1 seul
df = df.drop_duplicates(subset=['sku'])
df.reset_index(drop=True, inplace=True)

print("Produits après suppression des doublons :", len(df))

Produits après suppression des doublons : 130


# ÉTAPE 5 : Encodage et normalisation

In [35]:
# Encodage des variables catégorielles
encoded_cat = pd.get_dummies(df[['category', 'marque']])

# Normalisation des variables numériques
scaler = MinMaxScaler()
encoded_num = pd.DataFrame(
    scaler.fit_transform(df[['prix', 'note']]),
    columns=['prix', 'note']
)

# Fusion des features
X = pd.concat([encoded_cat, encoded_num], axis=1)

# ÉTAPE 6 : Calcul de la similarité cosinus

In [36]:
similarity_matrix = cosine_similarity(X)


# ÉTAPE 7 : Génération des recommandations UNIQUES

In [37]:
TOP_N = 5
recommendations = []

for idx in df.index:
    scores = list(enumerate(similarity_matrix[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    similar_products = []
    seen_sku = set()  # éviter les répétitions

    for i, _ in scores[1:]:  # ignorer le produit lui-même
        sku = df.loc[i, 'sku']

        if sku not in seen_sku:
            similar_products.append({
                "nom": df.loc[i, 'nom'],
                "marque": df.loc[i, 'marque'],
                "note": df.loc[i, 'note']
            })
            seen_sku.add(sku)

        if len(similar_products) == TOP_N:
            break

    recommendations.append({
        "sku": df.loc[idx, 'sku'],
        "nom": df.loc[idx, 'nom'],
        "marque": df.loc[idx, 'marque'],
        "note": df.loc[idx, 'note'],
        "recommandations": similar_products
    })

# ÉTAPE 8 : Sauvegarde des resultats

In [38]:
print("\n🔹 Exemple de recommandations :\n")
for rec in recommendations[:3]:
    print(rec)
    print("-" * 50)

rec_df = pd.DataFrame(recommendations)

rec_df.to_csv("product_recommendations.csv", index=False)
rec_df.to_json("product_recommendations.json", orient="records", indent=2)



🔹 Exemple de recommandations :

{'sku': '4e55f1f8-37a3-4ca8-80ac-471e1598804d', 'nom': 'CHAUSSURE COURSE A PIED HOMME RUN ONE BLEU', 'marque': 'DECATHLON', 'note': 10101.0, 'recommandations': [{'nom': 'CHAUSSETTES DE TENNIS HAUTES RS 500 NOIR LOT DE 3', 'marque': 'DECATHLON', 'note': 16136.0}, {'nom': 'CHAUSSETTES DE SPORT HAUTES  RS 160 NOIR LOT DE 3', 'marque': 'DECATHLON', 'note': 16669.0}, {'nom': 'CHAUSSETTES DE SPORT BASSES RS 160 BLANC LOT DE 3.', 'marque': 'DECATHLON', 'note': 21539.0}, {'nom': 'CHAUSSETTES DE SPORT BASSES RS 160 NOIR LOT DE 3.', 'marque': 'DECATHLON', 'note': 21539.0}, {'nom': 'CHAUSSETTES DE SPORT MI-HAUTES RS 160 NOIRES LOT DE 3.', 'marque': 'DECATHLON', 'note': 18418.0}]}
--------------------------------------------------
{'sku': '6ca4f924-20ab-4cec-9173-7af2f77c9443', 'nom': 'Chaussures de running homme jogflow 100.1 gris orange', 'marque': 'KALENJI', 'note': 14125.0, 'recommandations': [{'nom': 'Chaussettes de running run100 noire x3.', 'marque': 'KIPRUN